# Generic Rule Induction Notebook

Continuoulsy refined.

In [1]:
%store -z acc_list 

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [2]:
from config import config_dict

# document config order
CONFIG = config_dict["CONFIG15"]
print('Proceed with configuration:', CONFIG["NAME"])

[('CONFIG1', {'NAME': 'binary-churn-quantile-brcg', 'DATA_SET': '../data/churn_prob_out_35.csv', 'DATA_TYPES': {'Children': <class 'float'>, 'RatePlan': <class 'str'>}, 'DROP': ['Id', 'pChurn', '3_Class', '5_Class', 'is_test_set'], 'MODE': 'PREDICTIVE', 'TRAIN_TEST_SPLIT': 0.3, 'BINARIZER': 'QUANTILE', 'ALGO': 'BRCG', 'TARGET_LABEL': 'CHURN', 'TYPE': 'BINARY', 'EXAMPLE_FEATURE': 'Est Income', 'POS_CLASS': 'T', 'BASELINE': True, 'USECASE': None}), ('CONFIG2', {'NAME': 'continuous-churn', 'DATA_SET': '../data/churn_prob_out_35.csv', 'DATA_TYPES': {'Children': <class 'float'>, 'RatePlan': <class 'str'>}, 'DROP': ['Id', 'CHURN', '3_Class', '5_Class', 'is_test_set'], 'MODE': 'PREDICTIVE', 'TRAIN_TEST_SPLIT': 0.3, 'BINARIZER': 'QUANTILE', 'ALGO': 'GLRM', 'TARGET_LABEL': 'pChurn', 'TYPE': 'CONTINUOUS', 'EXAMPLE_FEATURE': 'Est Income', 'POS_CLASS': None, 'BASELINE': True, 'USECASE': None}), ('CONFIG3', {'NAME': 'bike-demand', 'DATA_SET': '../data/SeoulBikeData.csv', 'DATA_TYPES': {'Rented Bike

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import matthews_corrcoef,fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import wittgenstein as lw
import time
import warnings
import re

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREES':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper
if CONFIG['ALGO'] == 'CORELS':
    from corels import *



# from explainer import Explainer

# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


### Data

In [4]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG' or CONFIG['ALGO'] == 'CORELS' :
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)
    CONFIG['POS_CLASS'] = 1 
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df[CONFIG['TARGET_LABEL']].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   name                 100000 non-null  object 
 1   creditScore          100000 non-null  int64  
 2   income               100000 non-null  int64  
 3   loanAmount           100000 non-null  int64  
 4   monthDuration        100000 non-null  int64  
 5   rate                 100000 non-null  float64
 6   approval             100000 non-null  int64  
 7   yearlyReimbursement  100000 non-null  int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 6.1+ MB


0    78046
1    21954
Name: approval, dtype: int64

In [6]:
if CONFIG['TYPE'] == 'BINARY':
    print(df[CONFIG['TARGET_LABEL']].value_counts())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    df[CONFIG['TARGET_LABEL']].describe()
else:
    print('Unrecognized problem type')

0    78046
1    21954
Name: approval, dtype: int64


### Train, Test Split

In [7]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (70000, 7) (70000,)
Test: (30000, 7) (30000,)


### Reference Performance

In [8]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

### Binarization

In [9]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=False) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
#x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 76513 to 15795
Data columns (total 55 columns):
 #   Column                                         Non-Null Count  Dtype
---  ------                                         --------------  -----
 0   (name, , )                                     70000 non-null  int64
 1   (creditScore, <=, 355.0)                       70000 non-null  int32
 2   (creditScore, <=, 411.0)                       70000 non-null  int32
 3   (creditScore, <=, 466.0)                       70000 non-null  int32
 4   (creditScore, <=, 521.0)                       70000 non-null  int32
 5   (creditScore, <=, 577.0)                       70000 non-null  int32
 6   (creditScore, <=, 632.0)                       70000 non-null  int32
 7   (creditScore, <=, 686.0)                       70000 non-null  int32
 8   (creditScore, <=, 740.0)                       70000 non-null  int32
 9   (creditScore, <=, 795.0)                       70000 non-null  int32

feature   name creditScore                                                  \
operation               <=                                                   
value                355.0 411.0 466.0 521.0 577.0 632.0 686.0 740.0 795.0   
76513        0           1     1     1     1     1     1     1     1     1   
60406        0           0     0     0     0     0     0     0     0     1   
27322        0           0     0     1     1     1     1     1     1     1   
53699        0           0     0     0     0     1     1     1     1     1   
65412        0           0     0     0     0     0     0     1     1     1   

feature    ...                 rate yearlyReimbursement          \
operation  ...                   <=                  <=           
value      ... 0.057022398410936835  19616.800000000003 39038.0   
76513      ...                    0                   0       0   
60406      ...                    1                   1       1   
27322      ...                    0                   0       0   
53699      ...                    1                   0       0   
65412      ...                    1                   0       0   

feature                                                                   \
operation                                                                  
value     58746.200000000026 78681.0 98610.0 119176.80000000002 145325.9   
76513                      0       0       0                  0        0   
60406                      1       1       1                  1        1   
27322                      0       0       0                  0        1   
53699                      1       1       1                  1        1   
65412                      0       1       1                  1        1   

feature                                         
operation                                       
value     196541.00000000003 348566.9999999999  
76513                      1                 1  
60406                      1                 1  
27322                      1                 1  
53699                      1                 1  
65412                      1                 1  

[5 rows x 55 columns]

### Rule Induction

In [10]:
start_time = time.time()
print('Starting training for', CONFIG['ALGO'])

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    estimator = Ripper()
    estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    estimator = lw.RIPPER()
    estimator.fit(x_train_bin, y_train,class_feat=CONFIG["TARGET_LABEL"] , pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'CORELS':
    estimator = CorelsClassifier(n_iter=10000, 
                     max_card=2, # feautres per statement
                     c = 0.0001 # Higher values penalise longer rulelists
                    )
    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
    
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('Training time: ' + str(end_time - start_time))

Starting training for BRCG
Learning DNF rule with complexity parameters lambda0=0.001, lambda1=0.001
Initial LP solved
Iteration: 1, Objective: 0.1091
Iteration: 2, Objective: 0.1091
Training time: 7.616997718811035


In [12]:
CONFIG['POS_CLASS']

1

### Evaluation

In [13]:

if CONFIG['TYPE'] == 'BINARY':
    y_pred = estimator.predict(x_test_bin)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
    print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, y_pred))


   
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')
    


Accuracy: 0.8944666666666666
Balanced accuracy: 0.826851706303019
Precision: 0.7884975327548068
Recall: 0.7067256367241116
F1 0.7453755830786553
ConfusionMatrix [[22200  1243]
 [ 1923  4634]]
F-2 0.7216944401183616
Mathhews 0.680685397603254


In [14]:
#XGboost for Binary Classification
# storing the results in a list
acc_list = []
%store -r acc_list
if CONFIG['TYPE'] == 'BINARY' and CONFIG['BASELINE'] == True:
    
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_train_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_train_bin.columns.values]
    x_test_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_test_bin.columns.values]

    xgb_cl = xgb.XGBClassifier()
    xgb_cl.fit(x_train_bin, y_train)
    preds = xgb_cl.predict(x_test_bin)     
    print('Accuracy:', accuracy_score(y_test, preds))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, preds))
    print('Precision:', precision_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('F-2', fbeta_score(y_test, preds, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, preds))
    acc_list.append(accuracy_score(y_test, preds))
    %store acc_list

no stored variable or alias acc_list
Accuracy: 0.9826
Balanced accuracy: 0.9734873168592852
Precision: 0.962877741984967
Recall: 0.957297544608815
F-2 0.9584084038232509
Mathhews 0.9489618354410128
Stored 'acc_list' (list)


In [15]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)
elif CONFIG['ALGO'] == 'BRCG':
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
elif CONFIG['ALGO'] == 'RIPPER':
    print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    print('Rule set:')
    print(estimator.rule_list_to_pretty_string())
elif CONFIG['ALGO'] == 'CORELS':
    r_length = len(estimator.rl().rules)
    print("Rule Length:", r_length)
    for i in range(len(estimator.rl().rules[0]["antecedents"])):
        an = len(estimator.rl().rules[i]["antecedents"])
        print(f"Antecedents Length Rule {i}:" , an)
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    print("Rule Length:", len(estimator.ruleset_))

# uncomment the following line for a full optimized view of the model as data frame for GLRM rules
# explanation.style

Number of rules: 1
['loanAmount <= 997234.50 AND yearlyReimbursement <= 39038.00']


In [16]:
output = None
for label, rules in estimator.rule_map.items():
    print(type(rules))
    if label:
        output = rules
        break

        

AttributeError: 'BooleanRuleCG' object has no attribute 'rule_map'